# Python Catboost Tutorial - Regression

Adapted from the Catboost repository.

### CatBoost installation
If you have not already installed CatBoost: <br>
pip install --upgrade catboost


### Data Loading

In [1]:
from catboost import CatBoostRegressor, Pool, cv
from catboost.eval.catboost_evaluation import *

import numpy as np
import pandas as pd
from collections import Counter
from itertools import product

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from imblearn.over_sampling import SMOTE, SMOTENC

In [2]:
#Define function to calculate the MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
#Import Data
df = pd.read_csv("https://raw.githubusercontent.com/iandreafc/sna-bigdata-course/master/Datasets/titanic.csv")

#See the imported dataset
print("DF shape", df.shape)
df.head()


DF shape (891, 9)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


### Feature Preparation
First of all let's check how many missing values do we have:

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

As we cat see, **`Age`**, **`Cabin`** and **`Embarked`** indeed have some missing values, so let's fill them with some number way out of their distributions - so the model would be able to easily distinguish between them and take it into account:

In [5]:
df.fillna(-999, inplace=True)
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

Now let's separate features and label variable, **to test regression we try to predict the ticket price**:

In [6]:
X = df.drop('Fare', axis=1)
y = df.Fare

Pay attention that our features are of differnt types - some of them are numeric, some are categorical, and some are even just strings, which normally should be handled in some specific way (for example encoded with bag-of-words representation). But in our case we could treat these string features just as categorical one - all the heavy lifting is done inside CatBoost. How cool is that? :)

In [7]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != np.float)[0]
categorical_features_indices

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Cabin        object
Embarked     object
dtype: object


array([0, 1, 2, 4, 5, 6, 7], dtype=int64)

#### Encode Strings
Not strictly necessary in Catboost, but useful for example for SMOTE.

In [8]:
for var in ['Sex', 'Cabin', 'Embarked']:
    X[var] = X[var].astype('category').cat.codes
X.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,0,3,1,22.0,1,0,0,3
1,1,1,0,38.0,1,0,82,1
2,1,3,0,26.0,0,0,0,3
3,1,1,0,35.0,1,0,56,3
4,0,3,1,35.0,0,0,0,3


### Data Splitting
Let's split the train data into training and validation sets.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=14)

### Parameters Tuning

In [10]:
#Define a grid of parameters to test
grid = {'learning_rate': [0.01, 0.03, 0.1, 0.2],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        }

#Count all possible combinations
print("# Combinations:", len([dict(zip(grid.keys(),v)) for v in product(*grid.values())]))

# Combinations: 60


In [11]:
#Define Model
model = CatBoostRegressor()

#Use the Pool function to specify the categorical features
train_pool = Pool(data= X_train, label= y_train, cat_features= categorical_features_indices)

#Grid Search
#Default cross-validation is 3-fold
grid_search_result = model.grid_search(grid, train_pool, cv=3)
bestparam = grid_search_result["params"]
bestparam

0:	loss: 51.4257794	best: 51.4257794 (0)	total: 3.71s	remaining: 3m 38s
1:	loss: 48.5511426	best: 48.5511426 (1)	total: 7.91s	remaining: 3m 49s
2:	loss: 47.6622629	best: 47.6622629 (2)	total: 14.6s	remaining: 4m 36s
3:	loss: 46.5503441	best: 46.5503441 (3)	total: 21.6s	remaining: 5m 2s
4:	loss: 53.2308382	best: 46.5503441 (3)	total: 24.4s	remaining: 4m 28s
5:	loss: 44.8138739	best: 44.8138739 (5)	total: 27.9s	remaining: 4m 10s
6:	loss: 47.2186616	best: 44.8138739 (5)	total: 33.1s	remaining: 4m 10s
7:	loss: 42.1276775	best: 42.1276775 (7)	total: 39.5s	remaining: 4m 17s
8:	loss: 55.2727481	best: 42.1276775 (7)	total: 42.1s	remaining: 3m 58s
9:	loss: 47.4190118	best: 42.1276775 (7)	total: 44.9s	remaining: 3m 44s
10:	loss: 45.5511488	best: 42.1276775 (7)	total: 49.3s	remaining: 3m 39s
11:	loss: 45.0541410	best: 42.1276775 (7)	total: 55.6s	remaining: 3m 42s
12:	loss: 56.4005909	best: 42.1276775 (7)	total: 58s	remaining: 3m 29s
13:	loss: 51.9701344	best: 42.1276775 (7)	total: 1m	remaining: 3

{'depth': 6, 'l2_leaf_reg': 9, 'learning_rate': 0.1}

In [12]:
#Set best params
model = CatBoostRegressor()

#Depending on your objective you can also customize the evaluation metric
bestparam["eval_metric"] = "RMSE"

model.set_params(**bestparam)
print(model.get_params())

{'loss_function': 'RMSE', 'depth': 6, 'l2_leaf_reg': 9, 'learning_rate': 0.1, 'eval_metric': 'RMSE'}


### Model Training
Retaining the best model and with early stopping, to avoid overfit.
**In real cases, we need an external test set, not used for training or validation (early stopping). That dataset is the one to be used to evaluate the final moldel.**

In [13]:
#Furter split the train set into final_train and validation sets
X_train_final, X_validation, y_train_final, y_validation = train_test_split(X_train, y_train,\
                                                                            train_size=0.75, random_state=14)

print(X_train.shape, X_train_final.shape, X_validation.shape)

(668, 8) (501, 8) (167, 8)


Use early sotopping rounds and validation set, to stop after K iterations with no improvement of the evaluation metric.

In [14]:
model.fit(X_train_final, y_train_final, cat_features=categorical_features_indices,\
          eval_set=(X_validation, y_validation), early_stopping_rounds = 80,\
          use_best_model=True, logging_level = "Verbose")

0:	learn: 53.4933716	test: 36.0313344	best: 36.0313344 (0)	total: 43.2ms	remaining: 43.2s
1:	learn: 51.9761312	test: 34.4858794	best: 34.4858794 (1)	total: 50.1ms	remaining: 25s
2:	learn: 50.6086357	test: 33.1763546	best: 33.1763546 (2)	total: 64.7ms	remaining: 21.5s
3:	learn: 49.5492064	test: 32.1721324	best: 32.1721324 (3)	total: 73.1ms	remaining: 18.2s
4:	learn: 48.4987617	test: 31.0057940	best: 31.0057940 (4)	total: 84.1ms	remaining: 16.7s
5:	learn: 47.4528906	test: 30.1281782	best: 30.1281782 (5)	total: 90.5ms	remaining: 15s
6:	learn: 46.6617979	test: 29.7176684	best: 29.7176684 (6)	total: 99.2ms	remaining: 14.1s
7:	learn: 45.9528926	test: 29.3022339	best: 29.3022339 (7)	total: 113ms	remaining: 14s
8:	learn: 45.2723747	test: 28.6544811	best: 28.6544811 (8)	total: 120ms	remaining: 13.2s
9:	learn: 44.6471027	test: 28.1572765	best: 28.1572765 (9)	total: 126ms	remaining: 12.5s
10:	learn: 44.1531113	test: 27.6852475	best: 27.6852475 (10)	total: 129ms	remaining: 11.6s
11:	learn: 43.6548

With this we can see that the best **RMSE** value of **25.13** (on **validation set**) was acheived at step **27** with no futher improvement after **80** iterations (so the training stopped). We now retain this model as the **best model**.

### Model Predictions and Fit

In [15]:
#Predict on the original Test Set
predictions = model.predict(X_test)
truevalues = np.array(y_test)

#Calculate MAE, MAPE (if no 0 values) and RMSE
print("MAE:", '%.4f' % mean_absolute_error(truevalues, predictions))
print("MAPE:", '%.4f' % mean_absolute_percentage_error(truevalues, predictions))
print("RMSE:", '%.4f' %  np.sqrt(mean_squared_error(truevalues, predictions)))

#Replace zero (only for demonstrational purpose) and calculate MAPE
truevalues[truevalues == 0] = 1
print("\nMAPE(replaced zeros):", '%.4f' % mean_absolute_percentage_error(truevalues, predictions), "%")

#Compare with variable, just to get an idea
print("\ny_test M and SD (for comparison)")
print('%.4f' % y_test.mean(),'%.4f' % y_test.std())

MAE: 15.2911
MAPE: inf
RMSE: 29.2967

MAPE(replaced zeros): 130.7210 %

y_test M and SD (for comparison)
31.5728 44.1218


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


### Monte Carlo Cross-Validation
Now repeat the process 1,000 times and provide average fit statistics, with their standard deviation.

In [16]:
#Save accuracy and kappa scores in a list
MAE, RMSE = [], []

#For demonstrational purposes we now reapet it 10 times
for i in range(0,10):
    #Split with no random seed in train, validation and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)
    X_train_final, X_validation, y_train_final, y_validation = train_test_split(X_train, y_train, train_size=0.75)
    
    model.fit(X_train_final, y_train_final, cat_features=categorical_features_indices, \
              eval_set=(X_validation, y_validation), early_stopping_rounds = 80, use_best_model=True, \
              logging_level = "Silent")
    
    predictions = model.predict(X_test)
    truevalues = np.array(y_test)
    
    MAE.append(mean_absolute_error(truevalues, predictions))
    RMSE.append(np.sqrt(mean_squared_error(truevalues, predictions)))

In [17]:
print("MAE at each cross-validation step\n", MAE, "\n")
print("RMSE at each cross-validation step\n", RMSE, "\n")
print("MAE M", '%.4f' % np.mean(MAE), "SD", '%.4f' % np.std(MAE), "\n")
print("RMSE M", '%.4f' % np.mean(RMSE), "SD", '%.4f' % np.std(RMSE))

MAE at each cross-validation step
 [15.349770226229547, 12.050911218763286, 15.980185837429588, 18.39281760918325, 14.041911403186912, 14.597220030282347, 13.702341232634197, 13.474771506558714, 13.117398242733515, 12.528981643674657] 

RMSE at each cross-validation step
 [28.139670840662156, 29.341372292185817, 38.04193543636353, 60.782859370224884, 31.769903555965612, 43.880098554202554, 38.568208757964, 27.98350059775569, 22.679595556337027, 26.480237694644828] 

MAE M 14.3236 SD 1.7736 

RMSE M 34.7667 SD 10.6216
